In [368]:
import requests, bs4, pandas, re, datetime, numpy
from dateutil.parser import parse

In [ ]:
import requests,bs4

r = requests.get('https://southboroughcc.play-cricket.com/website/results/4462297')
r.raise_for_status()
soup = bs4.BeautifulSoup(r.content, 'html.parser')
r.text
rows = []
inningsno = r.text[r.text.find("innings")+len("innings"):r.text.find("-",r.text.find("innings"))]
for i in range(1,12):
    rows.append(soup.select('#innings'+ inningsno +' > div.table-responsive-sm > table.table.standm.table-hover > tbody > tr:nth-child(' + str(i) + ') > td:nth-child(4) > strong'))
## Extracts all batting scores within <strong>

In [ ]:
rows

In [875]:
str(r.text).find("innings")
str(r.text).find("-",24020)
str(r.text)[24034:24038]
r.text.find("innings")
r.text.find("-",r.text.find("innings"))
r.text[r.text.find("innings")+len("innings"):r.text.find("-",r.text.find("innings"))]
## Finds the innnings number as defined in Play-Cricket

NameError: name 'r' is not defined

In [ ]:
import requests,bs4

r2 = requests.get('https://southboroughcc.play-cricket.com/website/results/4462297')
r2.raise_for_status()
soup = bs4.BeautifulSoup(r2.content, 'html.parser')
elems = soup.select('#innings5411478 > div.table-responsive-sm > table.table.standm.table-hover > tbody > tr:nth-child(1)')
## batting scorecard table for first innings - nth child 1-11 are each batsman's row

elems[0].find_all('td')[0].find_all('div')[0].find_all('sup') ## find_all('td') finds all of the 'cells' of data in row. 

In [ ]:
import requests,bs4

r3 = requests.get('https://southboroughcc.play-cricket.com/website/results/4462297')
r3.raise_for_status()
soup3 = bs4.BeautifulSoup(r3.content, 'html.parser')
#elems3 = soup3.select('#innings5411478 > table > tbody') ##bowling scorecard for first innings
elems3 = soup3.select('#innings5411478 > table')
#elems3[0].find_all('td')



In [ ]:
import pandas
table = pandas.read_html(str(elems3[0]))[0] #Generates Panda table of bowling scorecard. Much easier to use than html extraction
table[["BOWLER","OVERSO"]]
len(table)
#for i in range(len(table)):
    #print(table.iloc[[i]])
#table.columns.values[1]
table.iloc[[3]].values[0]#[1]



In [ ]:
#### finds innings id's from match

def find_innings_id(string):
    sub = "innings"
    counter = 0
    inns_id = []
    
    while counter < len(string):
        if string.find(sub,counter) == -1:
            break
        begin = string.find(sub,counter)
        end = string.find('\"',begin)
        if "-" not in string[begin + len(sub):end] and string[begin + len(sub):end] not in inns_id:
            inns_id.append(string[begin + len(sub):end])
        counter = string.find(sub,counter) + len(sub)
    
    return inns_id
    

In [1276]:
### Overall match stats

##Extracts match point data [Home game, home penalty, home batting, home bowling, home total, away game, away penalty, away batting , away bowling, away total]

GamePoints_data = []

for i in range(1,3):
    GamePoints = soup4.select('#multiCollapseExample' + str(i) + ' > table')
    
    if len(GamePoints) == 0:
        GamePoints_data = ["NaN"]*10
        
    else:
        GamePoints_table = pandas.read_html(str(GamePoints[0]))[0]
        for j in range(len(GamePoints_table)):
            GamePoints_data.append(GamePoints_table.iloc[[j]].values[0][1])

print(GamePoints_data)        
        
##Extracts division and league data [league , division]

DivisionInfo = soup4.select('body > div.breadcrumb-league-wrapper > div.container.breadcrumb-league > div > div.col-sm-12.col-md-6.col-lg-6.text-center.text-lg-left.leaguedetail-left')
DivisionInfo_data = [""]*2

if 'friendly' in str(DivisionInfo[0]).lower():
    DivisionInfo_data = ["Friendly", "Friendly"]

elif DivisionInfo[0].find_all('span')[0].contents[0].strip() == "":
    DivisionInfo_data = [DivisionInfo[0].find_all('span')[2].contents[0].strip()]*2
    
elif 'https' in str(DivisionInfo[0].find_all('span')[2].contents[1]).lower():
    DivisionInfo_data[0] = DivisionInfo[0].find_all('span')[0].contents[0].strip()
    start = str(DivisionInfo[0].find_all('span')[2].contents[1]).find(">") + 1 
    end = str(DivisionInfo[0].find_all('span')[2].contents[1]).find("<",str(DivisionInfo[0].find_all('span')[2].contents[1]).find(">")) 
    DivisionInfo_data[1] = str(DivisionInfo[0].find_all('span')[2].contents[1])[start:end]
    
print(DivisionInfo_data)

##Extracts date and location data [ground , date]

DateGround_data = [""]*2

DateGround = soup4.select('body > div.breadcrumb-league-wrapper > div.container.breadcrumb-league > div > div.col-sm-12.col-md-6.col-lg-6.text-lg-right.leaguedetail-right')
Date = parse(re.search('\d*? \w* \d{4}',DateGround[0].text)[0])
Ground = DateGround[0].find_all('a')[0].contents[0]

if "\n" not in Ground:
    DateGround_data[0] = Ground
DateGround_data[1] = Date.date()

print(DateGround_data)



## Extracts [Home team, Away team, Home batting score, Away batting score, Home wickets lost, Away wickets lost, Home overs, Away overs, Winner, by runs, by wickets, other, toss winner, toss decision]

Teams = soup4.select('body > div.container.main-header.main-header-lg.d-none.d-lg-block > table')

Teams_data = [""]*14

for i in range(2):
    Clubname = Teams[0].find_all('p', class_="team-name")[i].contents[0]
    first = str(Teams[0].find_all('p', class_="team-info-2")[i].find_all(class_ = "team-info-1")[0]).find("\n")
    second = str(Teams[0].find_all('p', class_="team-info-2")[i].find_all(class_ = "team-info-1")[0]).find("\n", first + 1)
    Clubteam = str(Teams[0].find_all('p', class_="team-info-2")[i].find_all(class_ = "team-info-1")[0])[first+1:second].strip()
    Teams_data[i] = Clubname + ' ' + Clubteam


for i in range(2):
    first = str(Teams[0].find_all('p', class_ = "team-info-2")[i]).find("</span>\n")
    second = str(Teams[0].find_all('p', class_ = "team-info-2")[i]).find("<", first + 1)
    Battingscore = str(Teams[0].find_all('p', class_ = "team-info-2")[i])[first + len("</span>\n") : second].strip()
    Teams_data[2 + i] = Battingscore
    

for i in range(2):
    if len(Teams[0].find_all('p', class_ = "team-info-2")[i].find_all(class_ = "smalltxt")) == 0:
        Teams_data[4 + i], Teams_data[6 + i] = "",""
    else :
        WicketsOvers_html = Teams[0].find_all('p', class_ = "team-info-2")[i].find_all(class_ = "smalltxt")[0].contents[0]
        first = WicketsOvers_html.find("/")
        second = WicketsOvers_html.find("(", first + 1)
        third = WicketsOvers_html.find(")", second + 1)
        Wickets = WicketsOvers_html[first + 1 : second].strip()
        if "All out" in Wickets:
            Teams_data[4 + i] = 10
        else :
            Teams_data[4 + i] = Wickets
        Overs = WicketsOvers_html[second + 1 : third].strip()
        Teams_data[6 + i] = Overs

        
if "ABANDONED" in str(Teams[0]):
    Teams_data[8] = "None"
    Teams_data[11] = "Abandoned"

elif "CONCEDED" in str(Teams[0]):
    Teamconceded = Teams[0].find_all('p', class_ = "match-ttl win-cb-name")[0].contents[0].strip()
    if Teamconceded in Teams_data[0].upper():
        Teams_data[8] = Teams_data[1]
    else :
        Teams_data[8] = Teams_data[0]
    Teams_data[11] = "Conceded"
    
else :
    Matchwinner = Teams[0].find_all('p', class_ = "match-ttl win-cb-name")[0].contents[0].strip()
    first = str(Teams[0].find_all('div', class_ = "info mdont")[0].contents[1]).find("<span>")
    second = str(Teams[0].find_all('div', class_ = "info mdont")[0].contents[1]).find("<", first + 1)
    Wintype = str(Teams[0].find_all('div', class_ = "info mdont")[0].contents[1])[first + len("<span>") : second]
    Byhowmuch = ''.join(filter(str.isdigit, str(Teams[0].find_all('div', class_ = "info mdont")[0].contents[0])))

    if Matchwinner in Teams_data[0].upper():
        Teams_data[8] = Teams_data[0]
    else :
        Teams_data[8] = Teams_data[1]
    if Wintype == "RUNS":
            Teams_data[9] = Byhowmuch
    if Wintype == "WICKETS":
            Teams_data[10] = Byhowmuch      

            
if len(Teams[0].find_all('p', class_ = "team-info-3")) != 0:
    for i in range(2):
        if len(Teams[0].find_all('p', class_ = "team-info-3")[i].contents) != 0:
            Teams_data[12] = Teams_data[i]
            if "bat" in Teams[0].find_all('p', class_ = "team-info-3")[i].contents[0]:
                Teams_data[13] = "Bat"
            else :
                Teams_data[13] = "Field"

    if Teams_data[12] == "" and Teams_data[13] == "":
        Teams_data[12], Teams_data[13] = "No toss", "No toss"
        
elif len(Teams[0].find_all('p', class_ = "team-info-3 adma")) != 0:
    for i in range(2):
        if len(Teams[0].find_all('p', class_ = "team-info-3 adma")[i].contents) != 0:
            Teams_data[12] = Teams_data[i]
            if "bat" in Teams[0].find_all('p', class_ = "team-info-3 adma")[i].contents[0]:
                Teams_data[13] = "Bat"
            else :
                Teams_data[13] = "Field"

    if Teams_data[12] == "" and Teams_data[13] == "":
        Teams_data[12], Teams_data[13] = "No toss", "No toss"            
            
print(Teams_data)

## Extracts Playcricket match code (URL) and creates my own match reference

Reference_data = [""]*2

first = str(soup4).find('https://southboroughcc.play-cricket.com/website/results/')
second = str(soup4).find('\"',first + len('https://southboroughcc.play-cricket.com/website/results/'))
Reference_data[0] = str(soup4)[first + len('https://southboroughcc.play-cricket.com/website/results/') : second]

Date_reference = str(DateGround_data[1]).replace("-","")
Sboro_teams = ['Southborough CC 1st XI', 'Southborough CC 2nd XI', 'Southborough CC Sunday XI', 'Southborough CC Under 13', 'Southborough CC Under 13 B', 'Southborough CC Under 11']
Sboro_teams_references = ['1ST', '2ND', 'SUN', '13A', '13B', '11A']

for i in range(len(Sboro_teams)):
    if Teams_data[0] == Sboro_teams[i] or Teams_data[1] == Sboro_teams[i]:
        Team_reference = Sboro_teams_references[i]

Reference_data[1] = Team_reference + Date_reference

print(Reference_data)


['NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN']
['Kent County Village League', 'Kent County Village League']
['The Common', datetime.date(2006, 5, 27)]
['Southborough CC 1st XI', 'Sissinghurst CC 1st XI', '134', '130', '5', 10, '27.5', '37.1', 'Southborough CC 1st XI', '', '4', '', 'Southborough CC 1st XI', 'Field']
['542223', '1ST20060527']


In [1397]:
### extracts batting scorecard


for k in range(len(innings_id)):
    
    elems = soup4.select('body > div.container100sm.container.container-scorecard > div > div.col-sm-12.col-scorecard.mb50')
    if str(elems[0].find_all('a')[k].contents[0]).replace('\n', ' ').strip() in Teams_data[0]:
        batting_team = Teams_data[0]
        bowling_team = Teams_data[1]
    else:
        batting_team = Teams_data[1]
        bowling_team = Teams_data[0]
    print(batting_team)
    print(bowling_team)
    
    for i in range(11):
        
        print(i+1)
        
        elems = soup4.select('#innings' + innings_id[k] + ' > div.table-responsive-sm > table.table.standm.table-hover > tbody > tr:nth-child(' + str(1+i) + ') > td:nth-child(1) > div.bts')# > a')
        if len(elems) == 0:
            break
        if "Unsure" in elems[0]:
            print("Unsure")
        else:
            print(elems[0].contents[0].contents[0])
            if len(elems[0].contents[0].contents) == 2:
                start = str(elems[0].contents[0].contents[1]).find('img alt="')
                end = str(elems[0].contents[0].contents[1]).find('"', start + len('img alt="'))
                print(str(elems[0].contents[0].contents[1])[start + len('img alt="') : end])
            if len(elems[0].contents[0].contents) == 3:
                print('Keeper')
                print('Captain')

        elems = soup4.select('#innings' + innings_id[k] + ' > div.table-responsive-sm > table.table.standm.table-hover > tbody > tr:nth-child(' + str(1+i) + ') > td:nth-child(2)')
        if len(elems[0].contents) > 0:
            print(elems[0].contents[0].contents[0])
            if 'Unsure' in elems[0]:
                print('Unsure')
            elif len(elems[0].contents) > 1:
                print(elems[0].contents[1].contents[0])

        elems = soup4.select('#innings' + innings_id[k] + ' > div.table-responsive-sm > table.table.standm.table-hover > tbody > tr:nth-child(' + str(1+i) + ') > td:nth-child(3)')
        if len(elems[0].contents) > 0:
            print(elems[0].contents[0].contents[0].strip())
            if "Unsure" in elems[0]:
                print("Unsure")
            else:
                print(elems[0].contents[1].contents[0])

        elems = soup4.select('#innings' + innings_id[k] + ' > div.table-responsive-sm > table.table.standm.table-hover > tbody > tr:nth-child(' + str(1+i) + ') > td:nth-child(4)')
        if len(elems[0].contents) > 0:
            print(elems[0].contents[0].contents[0])

        for j in range(5,9):
            elems = soup4.select('#innings' + innings_id[k] + ' > div.table-responsive-sm > table.table.standm.table-hover > tbody > tr:nth-child(' + str(1+i) + ') > td:nth-child(' + str(j) + ')')
            if len(elems[0].contents) > 0:
                print(elems[0].contents[0])
                


Southborough CC 1st XI
Sissinghurst CC 1st XI
1
Phil Davis
Keeper
c
Tom Wild
b
Tom Dixon
22
0
5
0
2
Alex Juon
c
Arthur Sackville West
b
Tom Dixon
10
0
1
0
3
Tom Gillmor
c
Tom Dixon
b
Tom Wild
71
0
11
1
4
Chris Holder
ct & b
Tom Dixon
2
0
0
0
5
Ben Holt
b
Phil Clayton
5
0
1
0
6
Jack Roberts
c
Arthur Sackville West
b
Ben Dixon
53
0
7
3
7
Alex Hibbitt
b
Sebastian Guterres
0
0
0
0
8
Steve Smith
not out
13
0
1
0
9
Robert Berendt
b
Phil Clayton
10
0
0
1
10
Ben Hobden
did not bat
11
Stefan Hildreth
Captain
did not bat
Sissinghurst CC 1st XI
Southborough CC 1st XI
1
Arthur Sackville West
c
Phil Davis
b
Robert Berendt
69
62
8
1
111.29
2
Dan Callingham
Captain
c
Phil Davis
b
Robert Berendt
76
76
11
2
100.00
3
Rob Ede
not out
41
38
7
0
107.89
4
Julian Weekes
not out
23
20
4
0
115.00
5
William Sackville West
did not bat
6
Phil Clayton
did not bat
7
Tom Wild
did not bat
8
Tom Dixon
did not bat
9
Ben Dixon
did not bat
10
Alex Cribben
Keeper
did not bat
11
Sebastian Guterres
did not bat


In [1396]:
### Grabs batting extras data [1st byes, 1st leg byes, 1st wides, 1st no balls, 1st total, 2nd byes, etc]


Extras_data = [""]*10

for k in range(len(innings_id)):
    elems = soup4.select('#innings' + innings_id[k] + ' > div.table-responsive-sm > table.table.table-sm.table-scorecard-footer > tbody > tr:nth-child(1) > td.text-left.text-md-right.d-none.d-md-block > div')
    if elems[0].contents[1] == "0":
        Extras_data[5*k:5*k + 4] = [0]*4
    
    Extras_type = ["b", "lb", "w", "nb"]
    for i in range(4):
        if re.search('\d{1,2}' + Extras_type[i], elems[0].contents[1]) == None:
            Extras_data[5*k + i] = 0
        else:
            Extras_data[5*k + i] = int(re.search('\d{1,2}' + Extras_type[i], elems[0].contents[1])[0].replace(Extras_type[i], " ").strip())   
    Extras_data[5*k + 4] = sum(Extras_data[5*k : 5*k + 4])
            
Extras_data[4] = sum(Extras_data[0:4])
Extras_data[9] = sum(Extras_data[5:9])

print(Extras_data)



[9, 1, 5, 7, 22, 1, 1, 1, 0, 3]


In [1530]:
### Test webpages ####


#r4 = requests.get('https://southboroughcc.play-cricket.com/website/results/4462297') # Division / League
#r4 = requests.get('https://southboroughcc.play-cricket.com/website/results/4428817') # Friendly
#r4 = requests.get('https://southboroughcc.play-cricket.com/website/results/27378') # No division just league
#r4 = requests.get('https://southboroughcc.play-cricket.com/website/results/412586') # no division, no ground, abandoned
#r4 = requests.get('https://southboroughcc.play-cricket.com/website/results/4413968') # u13, conceded
#r4 = requests.get('https://southboroughcc.play-cricket.com/website/results/4462178') # team won tost and field first, and won by runs, and 10 fall of wickets with know batsmen / notout batsmen
#r4 = requests.get('https://southboroughcc.play-cricket.com/website/results/542223') # unsure batsmen, bowlers and dismissals
r4 = requests.get('https://southboroughcc.play-cricket.com/website/results/4512866') # 10 fall of wickets with unsure everything
#r4 = requests.get('https://southboroughcc.play-cricket.com/website/results/4516262') # 8 fall of wickets with unsure everything

r4.raise_for_status()
soup4 = bs4.BeautifulSoup(r4.content, 'html.parser')

innings_id = find_innings_id(r4.text)
print(innings_id)

['5427018', '5427017']


In [1531]:


for k in range(len(innings_id)):
    elems = soup4.select('#innings' + innings_id[k] + ' > div:nth-child(2) > div > p.font-3')
    FoW_teamruns = [""]*10
    FoW_batter = [""]*10
    FoW_NObatter = [""]*10
    FoW_NObatterscore = [""]*10
    if len(elems) > 0:
        for i in range(len(elems[0].contents)):
            
            #print(elems[0].contents[i])
            
            if re.search('\d{1,3}-\d{1,2}',str(elems[0].contents[i])) != None:
                wicketno = int(re.search('-\d{1,2}',str(elems[0].contents[i]))[0][1:])
                FoW_teamruns[wicketno - 1] = int(re.search('\d{1,3}-',str(elems[0].contents[i]))[0][:-1])
            
            if str(elems[0].contents[i]) == " (":
                FoW_batter[wicketno - 1] = str(elems[0].contents[i -1].contents[0])
                FoW_NObatter[wicketno - 1] = str(elems[0].contents[i + 1].contents[0])
                FoW_NObatterscore[wicketno - 1] = int(re.search('-\d{1,3}\*', str(elems[0].contents[i + 2]))[0][1:-1])
    
    print(FoW_teamruns)
    print(FoW_batter)
    print(FoW_NObatter)
    print(FoW_NObatterscore)                  
            


[52, 57, 57, 78, 165, 168, 171, 189, 189, 213]
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']


In [1528]:
FoW_teamruns = [""]*10
FoW_batter = [""]*10
FoW_NObatter = [""]*10
FoW_NObatterscore = [""]*10
    if str(elems[0].contents[3]) == " (":
        FoW_batter[wicketno - 1] = str(elems[0].contents[2].contents[0])
        FoW_NObatter[wicketno - 1] = str(elems[0].contents[4].contents[0])
        FoW_NObatterscore[wicketno - 1] = int(re.search('-\d{1,3}\*', str(elems[0].contents[5]))[0][1:-1])


print(FoW_batter)
print(FoW_NObatter)
print(FoW_NObatterscore)

IndentationError: unexpected indent (<ipython-input-1528-2f70c07d6f38>, line 5)

In [1523]:
int(re.search('-\d{1,3}\*', str(elems[0].contents[5]))[0][1:-1])

'15'